# Importing libraries

In [ ]:
import h5py as h5
import numpy as np
import mat73
import pickle
from os import path
import numpy as np
import sigpy as sp
import cupy
import sigpy.mri as mr
import sigpy.linop as lp
import sigpy.plot as pl
import tqdm as notebook_tqdm
import sigpy.plot as pl
import torch

# Ground truth and senstitivity map estimation from k-space data

In [ ]:
trnGT=[]
trnCsm=[]
#import torchvision.transforms as transforms

#transform = transforms.CenterCrop((64,64))
for i in range(1,17):
    filename= 'P'+ str(i)+'.mat'
    if(filename[-3:-1] =='ma'):  # mat file
        extension = '.pickle'
        fnamepickle = filename.replace('.mat',extension)
        if(not(path.exists(fnamepickle))):
            data_dict = mat73.loadmat(filename)
            kdata = data_dict['kSpace']
            #kdata = np.squeeze(kdata[:,:,:)
        # save with pickle for fast reading
            with open(fnamepickle, 'wb') as f:
                pickle.dump([kdata],f,protocol=4)
        else:
            with open(fnamepickle, 'rb') as f:
                [kdata] = pickle.load(f)
    else: # read pickle file
        fname = filename
        with open(fname, 'rb') as f:
            [kdata] = pickle.load(f)

    device = sp.Device(0)
    kdata = kdata.astype(np.complex64)
    kdata= np.transpose(kdata, (3,2,1,0))
    ksp = sp.to_device(kdata, device=0)
    mps = mr.app.EspiritCalib(ksp ,device=0).run()
    sens_maps = sp.to_device(mps, device=0)
    recon= mr.app.SenseRecon(ksp, sens_maps, device= 0).run()
    pl.ImagePlot(np.squeeze(mps[:,-1,:,:]),z=0,mode='m',colormap='jet',title='Log magnitude of k-space')
    pl.ImagePlot(np.squeeze(recon[-1,:,:]),mode='m',vmin=0, vmax=100,title='Log magnitude of k-space')
    trnGT.append(recon)
    trnCsm.append(mps)

# Concatenating along appropriate axis 

In [ ]:
Org=np.concatenate(trnGT, axis=0) 
Csm_0=np.concatenate(trnCsm, axis=1)
#trnOrg_LR= np.transpose(trnOrg_LR, (1,2,0))
Csm= np.transpose(Csm, (1, 0, 2, 3)) 
Csm.shape, Org.shape

 # Loading the sampling mask of R=4

In [ ]:
x= mat73.loadmat('mask_SR.mat')
trnMask=x['trnMask']
trnMask= trnMask.astype(int)

#tstMask=x['tstMask']
#tstMask= tstMask.astype(int)

#tstMask=np.transpose(tstMask, (2,0,1))
trnMask=np.transpose(trnMask, (2,0,1))
trnMask=np.fft.fftshift(trnMask)
trnMask.shape, #tstMask.shape

# Creating the *hdf5 dataset for MoDL training and testing

In [ ]:
with h5.File('modl_dataset_SR_size64_testing.hdf5', 'w') as f:
    f.create_dataset('/tstCsm', data=tstCsm_0.get())
    f.create_dataset('/tstMask', data=trnMask)
    f.create_dataset('/tstOrg', data=tstOrg.get())

# Creating *.png formatted to feed into SRResnet

In [ ]:
import imageio
for i in range(tstOrg.shape[0]):
    imageio.imwrite('/home/moalam/hpchome/Desktop/MoDL_package/SRGAN_dataset/HR_ground_truth_testing/'+ str(i)+'.png', np.squeeze(abs(tstOrg[i,:,:])).get())

# Checking *.hdf5 dataset dimension

In [ ]:
filename ="modl_dataset_SR_size64_testing.hdf5"
dataset = h5.File(filename, "r")
[print(item) for item in dataset.items()]
dataset.close()